# Código Ativações Placas
## Passo a passo

importando bibliotecas

In [6]:
import awswrangler as awr
import pandas as pd
import numpy as np
import datetime as dt
from extract import Extract
import logging
import traceback
import numpy as np

2 extraindo queries

In [7]:
class Extract:
    def __init__(self) -> None:
        pass
    
    def extract_ativacoes():

        try:

            dir_query = r'C:\Users\raphael.almeida\Documents\Ativações Placas\sql\placas_novas.sql'

            with open(dir_query, 'r') as file:
                query = file.read()

            df_ativ = awr.athena.read_sql_query(query,database='silver')
        
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('\n Relatorio ativacoes de placas novas  - Dados Extraidos com sucesso!')

            return df_ativ

        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'\n Falha ao Extrair relatorio ativacoes (Viavante): {e}')


    def extract_renovacoes():

        try:

            dir_query = r'C:\Users\raphael.almeida\Documents\Ativações Placas\sql\placas_renovadas.sql'

            with open(dir_query, 'r') as file:
                query = file.read()

            df_renov = awr.athena.read_sql_query(query, database='silver')
        
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('\n Relatorio renovacoes (Vivante)  - Dados Extraidos com sucesso!')

            return df_renov

        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'\n Falha ao Extrair relatorio renovacoes (Viavante): {e}')


    def extract_conf_migracoes():

        try:

            dir_query = r'C:\Users\raphael.almeida\Documents\Ativações Placas\sql\placas_total.sql'

            with open(dir_query, 'r') as file:
                query = file.read()

            df_migracoes_placas = awr.athena.read_sql_query(query, database='silver')
        
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('\n Relatorio renovacoes(Stcoop)  - Dados Extraidos com sucesso!')

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('\n Processo de Extracao de Dados concluido com sucesso!')

            return df_migracoes_placas

        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'\n Falha ao Extrair relatorio renovacoes (Stcoop): {e}')




3 transformando 

In [8]:
df_ativ = Extract.extract_ativacoes()
df_renov = Extract.extract_renovacoes()
df_mig_all_placas = Extract.extract_conf_migracoes()

In [21]:
# CRIANDO DATAFRAME FINAIL VIAVANTE (VERIFICANDO RENOVAÇÕES)
if not df_renov.empty:
    df_renov['status'] = 'RENOVAÇÃO'
    df_final = pd.concat([df_ativ, df_renov])

else:

    df_final = df_ativ

# CRIANDO DATAFRAME FINAIL VIAVANTE (VERIFICANDO MIGRAÇÕES)

# CRIANDO COLUNA DE MIGRAÇÃO (MIGRATION_FROM)
df_final['migration_from'] = None

print(len(df_renov))
print(len(df_ativ))
print(len(df_final))
df_final.head()
df_final['empresa'].unique()

42
4
46


<StringArray>
['Stcoop']
Length: 1, dtype: string

In [37]:
df_filtred = df_mig_all_placas[
    (df_mig_all_placas['placa'] == row['placa']) |
    (df_mig_all_placas['chassi'] == row['chassi'])
]

print(df_filtred)
print(len(df_mig_all_placas))

          placa             chassi  id_placa  matricula  conjunto   empresa
20095   QJM3E75  9BM958453KB132868     15162      21150      4813    Stcoop
21584   QJM3E75  9BM958453KB132868     15162      21150     13028    Stcoop
90738   QJM3E75  9BM958453KB132868     84594       5519     69662  Segtruck
107256  QJM3E75  9BM958453KB132868     84594       5519    121275  Segtruck
107257  QJM3E75  9BM958453KB132868     84594       5519     99470  Segtruck
223873


analisando df_filtred

In [39]:
df_filtred = pd.merge(df_mig_all_placas,df_final, on='placa', how='inner')
df_filtred['placa'].unique()

<StringArray>
['RLF8C01', 'QJM3E75', 'ATJ0C11', 'OHU4424', 'NCS2G02', 'NCS2F82', 'NCS2F32',
 'QZR5E18', 'KBY9G32', 'ALX8764', 'QZR5E28', 'QZR5D98', 'NCV0C13', 'MKX1D76',
 'MCW2I26', 'RZJ0F45', 'SFD4F88', 'RSX1B20', 'SEE1I94', 'OBI9G60', 'RSX1A90',
 'RSW9G20', 'RSX1B00', 'SEE1I95', 'SEE1I96', 'BAS9E39', 'SPH3H60', 'SPH3H78',
 'RSX1A60', 'RSW9G10', 'MEO1J44', 'PFK8F47', 'MBJ8F72', 'AZC4A17', 'DKW4D67',
 'DKW4D68', 'SFB5I74', 'SFB5J12', 'NJA9J69', 'HTS1F12', 'SFB2A36', 'SPH3H88',
 'ADO5011', 'BCG2774', 'SFB5J08', 'BCG2773']
Length: 46, dtype: string

In [34]:


# ATUALIZANDO MIGRAÇÕES DE PLACAS (SEGTRUCK & STCOOP ---> VIAVANTE)
for idx, row in df_final.iterrows():
# a iteração puxa linha a linha placas que constem na query geral de placas chamada "df_mig_all_placas", porém sua ordem é arbitrária
# ele só pega a primeira linha para determinar a origem da migração, o que é um equívoco 
# deve-se ordenar "df_mig_all_placas" pela data de atualização (checkado no athena: ele ordena por número de matrícula!!!)   
    df_filtred = df_mig_all_placas[
        (df_mig_all_placas['placa'] == row['placa']) |
        (df_mig_all_placas['chassi'] == row['chassi'])
    ]
# o df_filtred se repete entre todas as linhas com placas iguais (é um df volátil)
    if not df_filtred.empty and len(df_filtred['empresa'].values) > 1:

        if df_filtred['empresa'].values[1] != row['empresa']:
            df_final.at[idx, 'status'] = 'MIGRAÇÃO'

            if df_filtred['empresa'].values[1] == 'Segtruck':
                df_final.at[idx, 'migration_from'] = 'Segtruck'

            elif df_filtred['empresa'].values[1] == 'Stcoop':
                df_final.at[idx, 'migration_from'] = 'Stcoop'
            
            elif df_filtred['empresa'].values[1] == 'Stcoop':
                df_final.at[idx, 'migration_from'] = 'Viavante'
    else: 
        df_final.at[idx, 'migration_from'] = 'NULL' 

print(df_final.head())
print(df_final['migration_from'].unique())
print(len(df_final))





     placa             chassi  id_placa  id_veiculo  id_carroceria  matricula  \
0  DKW4D68  94BN075255R000270      8276           0           8276      19890   
1  MBJ8F72  93EL0262YY1000544      8278           0           8278      19890   
2  AZC4A17  93ZS3HUH0E8828134     13005       13005           <NA>      19890   
3  DKW4D67  94BN075255R000272      8277           0           8277      19890   
0  SEE1I96  97T0BN442N2008213      9831           0           9831        135   

   conjunto           unidade     status                     cliente  \
0     13160  UNIDADE CASCAVEL      ATIVO                 NERCI LANGE   
1     13160  UNIDADE CASCAVEL      ATIVO                 NERCI LANGE   
2     13160  UNIDADE CASCAVEL      ATIVO                 NERCI LANGE   
3     13160  UNIDADE CASCAVEL      ATIVO                 NERCI LANGE   
0     12983    UNIDADE CUIABA  RENOVAÇÃO  JOSILAINE DA SILVA ANDRADE   

        data data_ativacao                     suporte data_filtro empresa  \
0 

In [41]:



# TRANSFORMANDO O STATUS 'ATIVO' EM 'NOVO'
df_final.loc[df_final['status'] == 'ATIVO', 'status'] = 'NOVO'


    
df_final['placa'] = df_final['placa'].fillna('SEM-PLACA')
df_final['chassi'] = df_final['chassi'].fillna('NULL')
df_final['id_placa'] = df_final['id_placa'].fillna(0)
df_final['id_veiculo'] = df_final['id_veiculo'].fillna(0)
df_final['id_carroceria'] = df_final['id_carroceria'].fillna(0)
df_final['matricula'] = df_final['matricula'].fillna(0)
df_final['conjunto'] = df_final['conjunto'].fillna(0)
df_final['unidade'] = df_final['unidade'].fillna('NULL')
df_final['status'] = df_final['status'].fillna('NULL')
df_final['cliente'] = df_final['cliente'].fillna('NULL')
df_final['data'] = df_final['data'].fillna(pd.Timestamp('1900-01-01'))
df_final['data_ativacao'] = df_final['data_ativacao'].fillna(pd.Timestamp('1900-01-01'))
df_final['suporte'] = df_final['suporte'].fillna('NULL')
df_final['data_filtro'] = df_final['data_filtro'].fillna(pd.Timestamp('1900-01-01'))
df_final['empresa'] = df_final['empresa'].fillna('NULL')
df_final['migration_from'] = df_final['migration_from'].fillna('NULL')







analisando o load

In [44]:
import logging
import openpyxl




template = r'C:\Users\raphael.almeida\Documents\Ativações Placas\report\ACOMPANHAMENTO_ATIVACOES.xlsx'

wb = openpyxl.load_workbook(filename=template)

ws1 = wb['ATIVAÇÕES']







In [45]:
def get_last_row(sheet):
    for row in reversed(range(1, sheet.max_row + 1)):
        for col in range(1, sheet.max_column + 1):
            if sheet.cell(row=row, column=col).value is not None:
                return row
    return 1

In [49]:
# Encontrar a última linha não vazia em cada aba
last_row_ws1 = get_last_row(ws1)


wsdf = pd.DataFrame(ws1)
wsdf

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,<Cell 'ATIVAÇÕES'.A1>,<Cell 'ATIVAÇÕES'.B1>,<Cell 'ATIVAÇÕES'.C1>,<Cell 'ATIVAÇÕES'.D1>,<Cell 'ATIVAÇÕES'.E1>,<Cell 'ATIVAÇÕES'.F1>,<Cell 'ATIVAÇÕES'.G1>,<Cell 'ATIVAÇÕES'.H1>,<Cell 'ATIVAÇÕES'.I1>,<Cell 'ATIVAÇÕES'.J1>,<Cell 'ATIVAÇÕES'.K1>,<Cell 'ATIVAÇÕES'.L1>,<Cell 'ATIVAÇÕES'.M1>,<Cell 'ATIVAÇÕES'.N1>,<Cell 'ATIVAÇÕES'.O1>,<Cell 'ATIVAÇÕES'.P1>,<Cell 'ATIVAÇÕES'.Q1>
1,<Cell 'ATIVAÇÕES'.A2>,<Cell 'ATIVAÇÕES'.B2>,<Cell 'ATIVAÇÕES'.C2>,<Cell 'ATIVAÇÕES'.D2>,<Cell 'ATIVAÇÕES'.E2>,<Cell 'ATIVAÇÕES'.F2>,<Cell 'ATIVAÇÕES'.G2>,<Cell 'ATIVAÇÕES'.H2>,<Cell 'ATIVAÇÕES'.I2>,<Cell 'ATIVAÇÕES'.J2>,<Cell 'ATIVAÇÕES'.K2>,<Cell 'ATIVAÇÕES'.L2>,<Cell 'ATIVAÇÕES'.M2>,<Cell 'ATIVAÇÕES'.N2>,<Cell 'ATIVAÇÕES'.O2>,<Cell 'ATIVAÇÕES'.P2>,<Cell 'ATIVAÇÕES'.Q2>
2,<Cell 'ATIVAÇÕES'.A3>,<Cell 'ATIVAÇÕES'.B3>,<Cell 'ATIVAÇÕES'.C3>,<Cell 'ATIVAÇÕES'.D3>,<Cell 'ATIVAÇÕES'.E3>,<Cell 'ATIVAÇÕES'.F3>,<Cell 'ATIVAÇÕES'.G3>,<Cell 'ATIVAÇÕES'.H3>,<Cell 'ATIVAÇÕES'.I3>,<Cell 'ATIVAÇÕES'.J3>,<Cell 'ATIVAÇÕES'.K3>,<Cell 'ATIVAÇÕES'.L3>,<Cell 'ATIVAÇÕES'.M3>,<Cell 'ATIVAÇÕES'.N3>,<Cell 'ATIVAÇÕES'.O3>,<Cell 'ATIVAÇÕES'.P3>,<Cell 'ATIVAÇÕES'.Q3>
3,<Cell 'ATIVAÇÕES'.A4>,<Cell 'ATIVAÇÕES'.B4>,<Cell 'ATIVAÇÕES'.C4>,<Cell 'ATIVAÇÕES'.D4>,<Cell 'ATIVAÇÕES'.E4>,<Cell 'ATIVAÇÕES'.F4>,<Cell 'ATIVAÇÕES'.G4>,<Cell 'ATIVAÇÕES'.H4>,<Cell 'ATIVAÇÕES'.I4>,<Cell 'ATIVAÇÕES'.J4>,<Cell 'ATIVAÇÕES'.K4>,<Cell 'ATIVAÇÕES'.L4>,<Cell 'ATIVAÇÕES'.M4>,<Cell 'ATIVAÇÕES'.N4>,<Cell 'ATIVAÇÕES'.O4>,<Cell 'ATIVAÇÕES'.P4>,<Cell 'ATIVAÇÕES'.Q4>
4,<Cell 'ATIVAÇÕES'.A5>,<Cell 'ATIVAÇÕES'.B5>,<Cell 'ATIVAÇÕES'.C5>,<Cell 'ATIVAÇÕES'.D5>,<Cell 'ATIVAÇÕES'.E5>,<Cell 'ATIVAÇÕES'.F5>,<Cell 'ATIVAÇÕES'.G5>,<Cell 'ATIVAÇÕES'.H5>,<Cell 'ATIVAÇÕES'.I5>,<Cell 'ATIVAÇÕES'.J5>,<Cell 'ATIVAÇÕES'.K5>,<Cell 'ATIVAÇÕES'.L5>,<Cell 'ATIVAÇÕES'.M5>,<Cell 'ATIVAÇÕES'.N5>,<Cell 'ATIVAÇÕES'.O5>,<Cell 'ATIVAÇÕES'.P5>,<Cell 'ATIVAÇÕES'.Q5>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15455,<Cell 'ATIVAÇÕES'.A15456>,<Cell 'ATIVAÇÕES'.B15456>,<Cell 'ATIVAÇÕES'.C15456>,<Cell 'ATIVAÇÕES'.D15456>,<Cell 'ATIVAÇÕES'.E15456>,<Cell 'ATIVAÇÕES'.F15456>,<Cell 'ATIVAÇÕES'.G15456>,<Cell 'ATIVAÇÕES'.H15456>,<Cell 'ATIVAÇÕES'.I15456>,<Cell 'ATIVAÇÕES'.J15456>,<Cell 'ATIVAÇÕES'.K15456>,<Cell 'ATIVAÇÕES'.L15456>,<Cell 'ATIVAÇÕES'.M15456>,<Cell 'ATIVAÇÕES'.N15456>,<Cell 'ATIVAÇÕES'.O15456>,<Cell 'ATIVAÇÕES'.P15456>,<Cell 'ATIVAÇÕES'.Q15456>
15456,<Cell 'ATIVAÇÕES'.A15457>,<Cell 'ATIVAÇÕES'.B15457>,<Cell 'ATIVAÇÕES'.C15457>,<Cell 'ATIVAÇÕES'.D15457>,<Cell 'ATIVAÇÕES'.E15457>,<Cell 'ATIVAÇÕES'.F15457>,<Cell 'ATIVAÇÕES'.G15457>,<Cell 'ATIVAÇÕES'.H15457>,<Cell 'ATIVAÇÕES'.I15457>,<Cell 'ATIVAÇÕES'.J15457>,<Cell 'ATIVAÇÕES'.K15457>,<Cell 'ATIVAÇÕES'.L15457>,<Cell 'ATIVAÇÕES'.M15457>,<Cell 'ATIVAÇÕES'.N15457>,<Cell 'ATIVAÇÕES'.O15457>,<Cell 'ATIVAÇÕES'.P15457>,<Cell 'ATIVAÇÕES'.Q15457>
15457,<Cell 'ATIVAÇÕES'.A15458>,<Cell 'ATIVAÇÕES'.B15458>,<Cell 'ATIVAÇÕES'.C15458>,<Cell 'ATIVAÇÕES'.D15458>,<Cell 'ATIVAÇÕES'.E15458>,<Cell 'ATIVAÇÕES'.F15458>,<Cell 'ATIVAÇÕES'.G15458>,<Cell 'ATIVAÇÕES'.H15458>,<Cell 'ATIVAÇÕES'.I15458>,<Cell 'ATIVAÇÕES'.J15458>,<Cell 'ATIVAÇÕES'.K15458>,<Cell 'ATIVAÇÕES'.L15458>,<Cell 'ATIVAÇÕES'.M15458>,<Cell 'ATIVAÇÕES'.N15458>,<Cell 'ATIVAÇÕES'.O15458>,<Cell 'ATIVAÇÕES'.P15458>,<Cell 'ATIVAÇÕES'.Q15458>
15458,<Cell 'ATIVAÇÕES'.A15459>,<Cell 'ATIVAÇÕES'.B15459>,<Cell 'ATIVAÇÕES'.C15459>,<Cell 'ATIVAÇÕES'.D15459>,<Cell 'ATIVAÇÕES'.E15459>,<Cell 'ATIVAÇÕES'.F15459>,<Cell 'ATIVAÇÕES'.G15459>,<Cell 'ATIVAÇÕES'.H15459>,<Cell 'ATIVAÇÕES'.I15459>,<Cell 'ATIVAÇÕES'.J15459>,<Cell 'ATIVAÇÕES'.K15459>,<Cell 'ATIVAÇÕES'.L15459>,<Cell 'ATIVAÇÕES'.M15459>,<Cell 'ATIVAÇÕES'.N15459>,<Cell 'ATIVAÇÕES'.O15459>,<Cell 'ATIVAÇÕES'.P15459>,<Cell 'ATIVAÇÕES'.Q15459>


In [ ]:

# Adicionar dados em 'SEGTUCK & STCOOP'
for r_idx, row in enumerate(df_final.values, last_row_ws1 + 1): #df_final.values aqui é o que eu quero iterar, o start ultimalinha + 1 ( o last row não diz respeito ao df_final.values mas sim ao índice que será usado para colocar os novos valores = é uma planilha incremental)
    for c_idx, value in enumerate(row, 2):  # Coluna começa em coluna excel 2
        ws1.cell(row=r_idx, column=c_idx, value=value)

wb.save(r'C:\Users\raphael.almeida\Documents\Ativações Placas\report\ACOMPANHAMENTO_ATIVACOES.xlsx')
wb.close()


In [ ]:
import shutil
import os
import traceback            
# PASSANDO ARQUIVO PARA O DIRETÓRIO DO SHAREPOINT
            #file_path = r'C:\Users\raphael.almeida\Documents\Ativações Placas\report\ACOMPANHAMENTO_ATIVACOES.xlsx'
            #destination_dir  = r'C:\Users\raphael.almeida\Grupo Unus\analise de dados - Arquivos em excel'
            #destination_path = os.path.join(destination_dir, os.path.basename(file_path))
            #shutil.copy(file_path, destination_path)

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('\n Processo de Carregamento de Dados concluido com sucesso!')


if __name__ == '__main__':

    Load_relat_ativ_pend.load_files()